In [2]:
import datetime as dt
import numpy as np
import math
import matplotlib.pyplot as plt
from typing import Literal

from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import VecNormalize, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

from FleetRL.fleet_env.fleet_environment import FleetEnv

import pandas as pd
import pyomo.environ as pyo

In [3]:
# define parameters here for easier change
n_steps = 8600
n_episodes = 1
n_evs = 1
n_envs = 1
time_steps_per_hour = 4
use_case: str = "lmd"  # for file name
scenario: Literal["arb", "real"] = "arb"

In [4]:
env_kwargs = {"schedule_name": "lmd_sched_single_eval.csv",
              "building_name": "load_lmd.csv",
              "price_name": "spot_2021_new.csv",
              "tariff_name": "spot_2021_new_tariff.csv",
              "use_case": "lmd",
              "include_building": True,
              "include_pv": True,
              "time_picker": "static",
              "deg_emp": False,
              "include_price": True,
              "ignore_price_reward": False,
              "ignore_invalid_penalty": True,
              "ignore_overcharging_penalty": True,
              "ignore_overloading_penalty": False,
              "episode_length": n_steps,
              "normalize_in_env": False,
              "verbose": 0,
              "aux": True,
              "log_data": True,
              "calculate_degradation": True
              }

if scenario == "real":
    env_kwargs["spot_markup"] = 10
    env_kwargs["spot_mul"] = 1.5
    env_kwargs["feed_in_ded"] = 0.25
elif scenario == "arb":
    env_kwargs["spot_markup"] = 0
    env_kwargs["spot_mul"] = 1
    env_kwargs["feed_in_ded"] = 0

In [5]:
night_vec_env = make_vec_env(FleetEnv,
                             n_envs=n_envs,
                             vec_env_cls=SubprocVecEnv,
                             env_kwargs=env_kwargs)

In [6]:
night_norm_vec_env = VecNormalize(venv=night_vec_env,
                                  norm_obs=True,
                                  norm_reward=True,
                                  training=True,
                                  clip_reward=10.0)

In [7]:
env = FleetEnv(use_case=use_case,
               schedule_name=env_kwargs["schedule_name"],
               tariff_name=env_kwargs["tariff_name"],
               price_name=env_kwargs["price_name"],
               episode_length=n_steps,
               time_picker=env_kwargs["time_picker"],
               building_name=env_kwargs["building_name"])

In [8]:
# reading the input file as a pandas DataFrame
df: pd.DataFrame = env.db

# Extracting information from the df
ev_data = df["There"]
building_data = df["load"]  # building load in kW

if scenario == "arb":
    price_data = np.multiply(np.add(df["DELU"], env.ev_conf.fixed_markup), env.ev_conf.variable_multiplier) / 1000
    tariff_data = np.multiply(df["tariff"], 1-env.ev_conf.feed_in_deduction) / 1000
elif scenario == "real":
    price_data = np.multiply(np.add(df["DELU"], 0), 1) / 1000  # price in EUR/kWh
    tariff_data = np.multiply(np.add(df["DELU"], 0), 1) / 1000  # price in EUR/kWh

pv_data = df["pv"]  # pv power in kW
soc_on_return = df["SOC_on_return"]

battery_capacity = env.ev_conf.init_battery_cap  # EV batt size in kWh
p_trafo = env.load_calculation.grid_connection  # Transformer rating in kW

charging_eff = env.ev_conf.charging_eff  # charging losses
discharging_eff = env.ev_conf.discharging_eff  # discharging losses

init_soc = env.ev_conf.def_soc  # init SoC

evse_max_power = env.load_calculation.evse_max_power  # kW, max rating of the charger

In [9]:
# create pyomo model
model = pyo.ConcreteModel(name="sc_pyomo")

model.timestep = pyo.Set(initialize=range(len(df)))
model.time_batt = pyo.Set(initialize=range(0, len(df)+1))

In [10]:
# model parameters
model.building_load = pyo.Param(model.timestep, initialize={i: building_data[i] for i in range(len(df))})
model.pv = pyo.Param(model.timestep, initialize={i: pv_data[i] for i in range(len(df))})
model.ev_availability = pyo.Param(model.timestep, initialize={i: ev_data[i] for i in range(len(df))})
model.price = pyo.Param(model.timestep, initialize={i: price_data[i] for i in range(len(df))})
model.tariff = pyo.Param(model.timestep, initialize={i: tariff_data[i] for i in range(len(df))})

In [11]:
# decision variables
# this assumes only charging, I could also make bidirectional later
model.soc = pyo.Var(model.time_batt, bounds=(0, env.ev_conf.target_soc))
model.charging_signal = pyo.Var(model.timestep, within=pyo.NonNegativeReals, bounds=(0,1))
model.discharging_signal = pyo.Var(model.timestep, within=pyo.NonPositiveReals, bounds=(-1,0))
model.positive_action = pyo.Var(model.timestep, within=pyo.Binary)
model.used_pv = pyo.Var(model.timestep, within=pyo.NonNegativeReals)

In [12]:
def grid_limit(m, i):
    return ((m.charging_signal[i] + m.discharging_signal[i]) * evse_max_power
            + m.building_load[i] - m.pv[i] + m.used_pv[i]  <= p_trafo) # added the used_pv to trafo limit

In [13]:
def mutual_exclusivity_charging(m, i):
    return m.charging_signal[i] <= model.positive_action[i]

def mutual_exclusivity_discharging(m, i):
    return m.discharging_signal[i] >= (model.positive_action[i] - 1)

In [68]:
def pv_use(m, i):
    return m.used_pv[i] <= m.charging_signal[i] * evse_max_power

In [69]:
def pv_constraint(m, i):
    return m.used_pv[i] <= m.pv[i]

In [70]:
def no_charge_when_no_car(m, i):
    if m.ev_availability[i] == 0:
        return m.charging_signal[i] == 0
    else:
        return pyo.Constraint.Feasible

def no_discharge_when_no_car(m, i):
    if m.ev_availability[i] == 0:
        return m.discharging_signal[i] == 0
    else:
        return pyo.Constraint.Feasible

In [71]:
def soc_rules(m, i):
    #last time step
    if i == len(df)-1:
        return (m.soc[i+1]
                == m.soc[i] + (m.charging_signal[i]*charging_eff + m.discharging_signal[i])
                * evse_max_power * 1 / time_steps_per_hour / battery_capacity)

    # new arrival
    elif (m.ev_availability[i] == 0) and (m.ev_availability[i+1] == 1):
        return m.soc[i+1] == soc_on_return[i+1]

    # departure in next time step
    elif (m.ev_availability[i] == 1) and (m.ev_availability[i+1] == 0):
        return m.soc[i] == env.ev_conf.target_soc

    else:
        return pyo.Constraint.Feasible

In [72]:
def charging_dynamics(m, i):
    #last time step
    if i == len(df)-1:
        return (m.soc[i+1]
                == m.soc[i] + (m.charging_signal[i]*charging_eff + m.discharging_signal[i])
                * evse_max_power * 1 / time_steps_per_hour / battery_capacity)

    # charging
    if (m.ev_availability[i] == 1) and (m.ev_availability[i+1] == 1):
        return (m.soc[i+1]
                == m.soc[i] + (m.charging_signal[i]*charging_eff + m.discharging_signal[i])
                * evse_max_power * 1 / time_steps_per_hour / battery_capacity)

    elif (m.ev_availability[i] == 1) and (m.ev_availability[i+1] == 0):
        return m.soc[i+1] == 0

    elif m.ev_availability[i] == 0:
        return m.soc[i] == 0

    else:
        return pyo.Constraint.Feasible

In [73]:
def max_charging_limit(m, i):
    return m.charging_signal[i]*evse_max_power <= evse_max_power * ev_data[i]

def max_discharging_limit(m, i):
    return m.discharging_signal[i]*evse_max_power*-1 <= evse_max_power * ev_data[i]

In [74]:
def first_soc(m):
    return m.soc[0] == init_soc

In [75]:
# constraints
model.cs1 = pyo.Constraint(rule=first_soc)
model.cs2 = pyo.Constraint(model.timestep, rule=grid_limit)
model.cs3 = pyo.Constraint(model.timestep, rule=max_charging_limit)
model.cs4 = pyo.Constraint(model.timestep, rule=max_discharging_limit)
model.cs5 = pyo.Constraint(model.timestep, rule=soc_rules)
model.cs6 = pyo.Constraint(model.timestep, rule=charging_dynamics)
model.cs8 = pyo.Constraint(model.timestep, rule=mutual_exclusivity_charging)
model.cs9 = pyo.Constraint(model.timestep, rule=mutual_exclusivity_discharging)
model.cs10 = pyo.Constraint(model.timestep, rule=no_charge_when_no_car)
model.cs11 = pyo.Constraint(model.timestep, rule=no_discharge_when_no_car)
model.cs12 = pyo.Constraint(model.timestep, rule=pv_use)
model.cs13 = pyo.Constraint(model.timestep, rule=pv_constraint)

In [76]:
timestep_set = pyo.RangeSet(0, len(df)-1)

In [77]:
def obj_fun(m):
    return (sum([((m.charging_signal[i] * evse_max_power - m.used_pv[i]) / time_steps_per_hour) * m.price[i] +
                 ((m.discharging_signal[i] * evse_max_power * discharging_eff) / time_steps_per_hour) * m.tariff[i]
                 for i in m.timestep]))

model.obj = pyo.Objective(rule=obj_fun, sense=pyo.minimize)

In [78]:
opt = pyo.SolverFactory('glpk', keepfiles=True, tmpdir='./tmp_glpk/')
res = opt.solve(model, tee=True)
print(res)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpiya1mfqr.glpk.raw --wglp /tmp/tmpb7qwotle.glpk.glp --cpxlp
 /tmp/tmp88zvnrhf.pyomo.lp
Reading problem data from '/tmp/tmp88zvnrhf.pyomo.lp'...
/tmp/tmp88zvnrhf.pyomo.lp:1754126: warning: lower bound of variable 'x280952' redefined
/tmp/tmp88zvnrhf.pyomo.lp:1754126: warning: upper bound of variable 'x280952' redefined
309210 rows, 175201 columns, 546201 non-zeros
35040 integer variables, all of which are binary
1789166 lines were read
Writing problem data to '/tmp/tmpb7qwotle.glpk.glp'...
1521767 lines were written
GLPK Integer Optimizer 5.0
309210 rows, 175201 columns, 546201 non-zeros
35040 integer variables, all of which are binary
Preprocessing...
68349 rows, 88944 columns, 177265 non-zeros
20909 integer variables, all of which are binary
Scaling...
 A: min|aij| =  4.171e-02  max|aij| =  1.100e+01  ratio =  2.637e+02
GM: min|aij| =  8.839e-01  max|aij| =  1.131e+00  ratio =  1.280e+00
EQ: min

Process ForkServerProcess-3:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/enzo/Desktop/FleetRL/venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 34, in _worker
    cmd, data = remote.recv()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 414, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [79]:
actions = [model.charging_signal[i].value + model.discharging_signal[i].value for i in range(len(df))]

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:
plt.plot(sorted(actions))
plt.show()

In [ ]:
actions = pd.DataFrame({"action": actions})

In [ ]:
actions.index = pd.date_range(start="2020-01-01 00:00", end="2020-12-30 23:59", freq="15T")

In [ ]:
for i in range(len(actions)):
    actions.loc[i, "hid"] = actions.index[i].hour + actions.index[i].minute/60

In [ ]:
actions.groupby("hid").mean().reset_index().plot()
plt.show()

In [15]:

import datetime as dt
import numpy as np
import math
import matplotlib.pyplot as plt
from typing import Literal

from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import VecNormalize, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

from FleetRL.fleet_env.fleet_environment import FleetEnv

import pandas as pd
import pyomo.environ as pyo
#import gurobipy

if __name__ == "__main__":

    # define parameters here for easier change
    n_steps = 8600
    n_episodes = 1
    n_evs = 1
    n_envs = 1
    time_steps_per_hour = 4
    use_case: str = "lmd"  # for file name
    scenario: Literal["arb", "real"] = "arb"
    
    # environment arguments
    env_kwargs = {"schedule_name": "lmd_sched_single_eval.csv",
                  "building_name": "load_lmd.csv",
                  "price_name": "spot_2021_new.csv",
                  "tariff_name": "spot_2021_new_tariff.csv",
                  "use_case": "lmd",
                  "include_building": True,
                  "include_pv": True,
                  "time_picker": "static",
                  "deg_emp": False,
                  "include_price": True,
                  "ignore_price_reward": False,
                  "ignore_invalid_penalty": True,
                  "ignore_overcharging_penalty": True,
                  "ignore_overloading_penalty": False,
                  "episode_length": n_steps,
                  "normalize_in_env": False,
                  "verbose": 0,
                  "aux": True,
                  "log_data": True,
                  "calculate_degradation": True
                  }
    
    if scenario == "real":
        env_kwargs["spot_markup"] = 10
        env_kwargs["spot_mul"] = 1.5
        env_kwargs["feed_in_ded"] = 0.25
    elif scenario == "arb":
        env_kwargs["spot_markup"] = 0
        env_kwargs["spot_mul"] = 1
        env_kwargs["feed_in_ded"] = 0
    
    lin_vec_env = make_vec_env(FleetEnv,
                               n_envs=n_envs,
                               vec_env_cls=SubprocVecEnv,
                               env_kwargs=env_kwargs)
    
    lin_norm_vec_env = VecNormalize(venv=lin_vec_env,
                                    norm_obs=True,
                                    norm_reward=True,
                                    training=True,
                                    clip_reward=10.0)
    
    env = FleetEnv(use_case=use_case,
                   schedule_name=env_kwargs["schedule_name"],
                   tariff_name=env_kwargs["tariff_name"],
                   price_name=env_kwargs["price_name"],
                   episode_length=n_steps,
                   time_picker=env_kwargs["time_picker"],
                   building_name=env_kwargs["building_name"])
    
    # reading the input file as a pandas DataFrame
    df: pd.DataFrame = env.db
    
    # Extracting information from the df
    ev_data = df["There"]
    building_data = df["load"]  # building load in kW
    
    if scenario == "arb":
        price_data = np.multiply(np.add(df["DELU"], env.ev_conf.fixed_markup), env.ev_conf.variable_multiplier) / 1000
        tariff_data = np.multiply(df["tariff"], 1-env.ev_conf.feed_in_deduction) / 1000
    elif scenario == "real":
        price_data = np.multiply(np.add(df["DELU"], 0), 1) / 1000  # price in EUR/kWh
        tariff_data = np.multiply(np.add(df["DELU"], 0), 1) / 1000  # price in EUR/kWh
    
    pv_data = df["pv"]  # pv power in kW
    soc_on_return = df["SOC_on_return"]
    
    battery_capacity = env.ev_conf.init_battery_cap  # EV batt size in kWh
    p_trafo = env.load_calculation.grid_connection  # Transformer rating in kW
    
    charging_eff = env.ev_conf.charging_eff  # charging losses
    discharging_eff = env.ev_conf.discharging_eff  # discharging losses
    
    init_soc = env.ev_conf.def_soc  # init SoC
    
    evse_max_power = env.load_calculation.evse_max_power  # kW, max rating of the charger
    
    # create pyomo model
    model = pyo.ConcreteModel(name="sc_pyomo")
    
    model.timestep = pyo.Set(initialize=range(len(df)))
    model.time_batt = pyo.Set(initialize=range(0, len(df)+1))
    
    # model parameters
    model.building_load = pyo.Param(model.timestep, initialize={i: building_data[i] for i in range(len(df))})
    model.pv = pyo.Param(model.timestep, initialize={i: pv_data[i] for i in range(len(df))})
    model.ev_availability = pyo.Param(model.timestep, initialize={i: ev_data[i] for i in range(len(df))})
    model.price = pyo.Param(model.timestep, initialize={i: price_data[i] for i in range(len(df))})
    model.tariff = pyo.Param(model.timestep, initialize={i: tariff_data[i] for i in range(len(df))})
    
    # decision variables
    # this assumes only charging, I could also make bidirectional later
    model.soc = pyo.Var(model.time_batt, bounds=(0, env.ev_conf.target_soc))
    model.charging_signal = pyo.Var(model.timestep, within=pyo.NonNegativeReals, bounds=(0,1))
    model.discharging_signal = pyo.Var(model.timestep, within=pyo.NonPositiveReals, bounds=(-1,0))
    model.positive_action = pyo.Var(model.timestep, within=pyo.Binary)
    model.used_pv = pyo.Var(model.timestep, within=pyo.NonNegativeReals)
    
    def grid_limit(m, i):
        return ((m.charging_signal[i] + m.discharging_signal[i]) * evse_max_power
                + m.building_load[i] - m.pv[i] <= p_trafo)
    
    def mutual_exclusivity_charging(m, i):
        return m.charging_signal[i] <= model.positive_action[i]
    
    def mutual_exclusivity_discharging(m, i):
        return m.discharging_signal[i] >= (model.positive_action[i] - 1)
    
    def pv_use(m, i):
        return m.used_pv[i] <= m.charging_signal[i] * evse_max_power
    
    def pv_constraint(m, i):
        return m.used_pv[i] <= m.pv[i]
    
    def no_charge_when_no_car(m, i):
        if m.ev_availability[i] == 0:
            return m.charging_signal[i] == 0
        else:
            return pyo.Constraint.Feasible
    
    def no_discharge_when_no_car(m, i):
        if m.ev_availability[i] == 0:
            return m.discharging_signal[i] == 0
        else:
            return pyo.Constraint.Feasible
    
    def soc_rules(m, i):
        #last time step
        if i == len(df)-1:
            return (m.soc[i+1]
                    == m.soc[i] + (m.charging_signal[i]*charging_eff + m.discharging_signal[i])
                    * evse_max_power * 1 / time_steps_per_hour / battery_capacity)
    
        # new arrival
        elif (m.ev_availability[i] == 0) and (m.ev_availability[i+1] == 1):
            return m.soc[i+1] == soc_on_return[i+1]
    
        # departure in next time step
        elif (m.ev_availability[i] == 1) and (m.ev_availability[i+1] == 0):
            return m.soc[i] == env.ev_conf.target_soc
    
        else:
            return pyo.Constraint.Feasible
    
    def charging_dynamics(m, i):
        #last time step
        if i == len(df)-1:
            return (m.soc[i+1]
                    == m.soc[i] + (m.charging_signal[i]*charging_eff + m.discharging_signal[i])
                    * evse_max_power * 1 / time_steps_per_hour / battery_capacity)
    
        # charging
        if (m.ev_availability[i] == 1) and (m.ev_availability[i+1] == 1):
            return (m.soc[i+1]
                    == m.soc[i] + (m.charging_signal[i]*charging_eff + m.discharging_signal[i])
                    * evse_max_power * 1 / time_steps_per_hour / battery_capacity)
    
        elif (m.ev_availability[i] == 1) and (m.ev_availability[i+1] == 0):
            return m.soc[i+1] == 0
    
        elif m.ev_availability[i] == 0:
            return m.soc[i] == 0
    
        else:
            return pyo.Constraint.Feasible
    
    def max_charging_limit(m, i):
        return m.charging_signal[i]*evse_max_power <= evse_max_power * ev_data[i]
    
    def max_discharging_limit(m, i):
        return m.discharging_signal[i]*evse_max_power*-1 <= evse_max_power * ev_data[i]
    
    def first_soc(m):
        return m.soc[0] == init_soc
    
    # constraints
    model.cs1 = pyo.Constraint(rule=first_soc)
    model.cs2 = pyo.Constraint(model.timestep, rule=grid_limit)
    model.cs3 = pyo.Constraint(model.timestep, rule=max_charging_limit)
    model.cs4 = pyo.Constraint(model.timestep, rule=max_discharging_limit)
    model.cs5 = pyo.Constraint(model.timestep, rule=soc_rules)
    model.cs6 = pyo.Constraint(model.timestep, rule=charging_dynamics)
    model.cs8 = pyo.Constraint(model.timestep, rule=mutual_exclusivity_charging)
    model.cs9 = pyo.Constraint(model.timestep, rule=mutual_exclusivity_discharging)
    model.cs10 = pyo.Constraint(model.timestep, rule=no_charge_when_no_car)
    model.cs11 = pyo.Constraint(model.timestep, rule=no_discharge_when_no_car)
    model.cs12 = pyo.Constraint(model.timestep, rule=pv_use)
    model.cs13 = pyo.Constraint(model.timestep, rule=pv_constraint)
    
    timestep_set = pyo.RangeSet(0, len(df)-1)
    
    def obj_fun(m):
        return (sum([((m.charging_signal[i] * evse_max_power - m.used_pv[i]) / time_steps_per_hour) * m.price[i] +
                     ((m.discharging_signal[i] * evse_max_power * discharging_eff) / time_steps_per_hour) * m.tariff[i]
                     for i in m.timestep]))
    
    model.obj = pyo.Objective(rule=obj_fun, sense=pyo.minimize)
    
    opt = pyo.SolverFactory('gurobi', solver_io="python")#, executable="/home/enzo/Downloads/gurobi10.0.2_linux64/gurobi1002/linux64/")

    res = opt.solve(model, tee=True)
    print(res)

    actions = [model.charging_signal[i].value + model.discharging_signal[i].value for i in range(len(df))]
    
    actions = pd.DataFrame({"action": actions})
    
    actions.index = pd.date_range(start="2020-01-01 00:00", end="2020-12-30 23:59", freq="15T")

    actions["hid"] = actions.index.dt.hour + actions.index.dt.minute/60
    
    actions.groupby("hid").mean().reset_index().plot()
    plt.show()

    start_time = lin_norm_vec_env.env_method("get_start_time")[0]
    end_time = start_time + dt.timedelta(hours=n_steps)
    env_actions = actions.loc[(actions.index >= start_time) & (actions.index <= end_time)]

    lin_norm_vec_env.reset()

    for i in range(n_steps*time_steps_per_hour):
        env.step([np.ones(n_evs) * env_actions[i]])

    lin_log: pd.DataFrame = lin_norm_vec_env.env_method("get_log")[0]

    lin_log.reset_index(drop=True, inplace=True)
    lin_log = lin_log.iloc[0:-2]

    # night_log.to_csv(f"log_dumb_{use_case}_{n_evs}.csv")
    real_power_lin = []
    for i in range(lin_log.__len__()):
        lin_log.loc[i, "hour_id"] = (lin_log.loc[i, "Time"].hour + lin_log.loc[i, "Time"].minute / 60)

    mean_per_hid_lin = lin_log.groupby("hour_id").mean()["Charging energy"].reset_index(drop=True)
    mean_all_lin = []
    for i in range(mean_per_hid_lin.__len__()):
        mean_all_lin.append(np.mean(mean_per_hid_lin[i]))

    mean = pd.DataFrame()
    mean["Distributed charging"] = np.multiply(mean_all_lin, 4)

    mean.plot()

    plt.xticks([0,8,16,24,32,40,48,56,64,72,80,88]
               ,["00:00","02:00","04:00","06:00","08:00","10:00","12:00","14:00","16:00","18:00","20:00","22:00"],
               rotation=45)

    plt.legend()
    plt.grid(alpha=0.2)

    plt.ylabel("Charging power in kW")
    max = lin_log.loc[0, "Observation"][-10]
    plt.ylim([-max * 1.2, max * 1.2])

    plt.show()



ModuleNotFoundError: No module named 'gurobipy'


ApplicationError: No Python bindings available for <class 'pyomo.solvers.plugins.solvers.gurobi_direct.GurobiDirect'> solver plugin

In [16]:
%cd /home/enzo/Downloads/gurobi10.0.2_linux64/gurobi1002/linux64/

/home/enzo/Downloads/gurobi10.0.2_linux64/gurobi1002/linux64


In [19]:
pip install -e .

Obtaining file:///home/enzo/Downloads/gurobi10.0.2_linux64/gurobi1002/linux64
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gurobipy
    Found existing installation: gurobipy 10.0.2
    Uninstalling gurobipy-10.0.2:
      Successfully uninstalled gurobipy-10.0.2
  Running setup.py develop for gurobipy

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
